<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/1_3_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://sitejerk.com/images/google-earth-logo-png-5.png" width=5% >
<img align="right" style="padding-left:10px;" src="https://colab.research.google.com/img/colab_favicon_256px.png" width=6% >


>> *This notebook is part from the free course [EEwPython](https://github.com/csaybar/EEwPython); the content is available [on GitHub](https://github.com/csaybar/EEwPython)* and released under the [Apache 2.0 License](https://www.gnu.org/licenses/gpl-3.0.en.html).



<!--NAVIGATION-->
 < [Data structures in GEE](1.2_DataStructures.ipynb) | [Contents](index.ipynb) |  [Feature & FeatureCollections](1.4_Features.ipynb)>

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/1.3_Images.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

In [0]:
#@title # ee.Image & ee.ImageCollection
from IPython.display import HTML
HTML('<center><iframe width="560" height="315" src="https://www.youtube.com/embed/LzxQH0Ze0iI" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></center>')

Raster data are represented as Image objects in Earth Engine. Images are composed of one or more bands and each band has its own name, data type, scale, mask and projection. Each image can have metadata stored as a set of properties (Dictionary).

In [0]:
#@title Function to visualize thumbnail URL
from IPython.display import Image
from IPython.display import HTML

def display_image(url,title="",CSS_row=False):
  img_link = "<center> <h1>%s</h1> <img title='%s' src= '%s'></center>" % (title,title,url)
  return HTML(img_link)

### Connecting GEE, Colab and Google Drive

- **Colab & Google Drive synchronization**

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

- **Colab & Earth Engine synchronization**

In [0]:
!pip install earthengine-api #Firstly we need the earth-engine API

In [0]:
!earthengine authenticate 

In [0]:
import ee
ee.Initialize()

### 2. Images overview

In addition to loading images from the archive by an **image ID**, you can also create images from constants, lists or other suitable Earth Engine objects. This section illustrates methods for creating images, getting band subsets, and manipulating bands.

Methods that you will use in this seccion:
 - **ee.Image.cat**
 - **ee.Image.addBands**
 - **ee.Image.Select**

In [0]:
# 2.1 Create a constant image.
image1 = ee.Image(1)
print(image1,'\n')
image1.getInfo()

- **ee.Image.cat(list)**: Concatenate the given images together into a single image.

  

In [0]:
#2.2 Concatenate two images into one multi-band image.
image2 = ee.Image(2)
image3 = ee.Image.cat([image1, image2])
print(image3,'\n')
image3.getInfo()

In [0]:
# Create a multi-band image from a list of constants.
multiband = ee.Image([1, 2, 3])
print(multiband,'\n')
multiband.getInfo()

In [0]:
# Select and (optionally) rename bands.
renamed = multiband.select(['constant', 'constant_1', 'constant_2'],
                           ['band1', 'band2', 'band3'])
print(renamed)

- **ee.Image.addBands**: Returns an image containing all bands copied from the first input and selected bands from the second input, optionally overwriting bands in the first image with the same name. The new image has the metadata and footprint from the first input image

In [0]:
# Add bands to an image.
image4 = image3.addBands(ee.Image(42))
print(image4)

### 3. Image Vizualization

The Python API  a difference than the Code Editor do not support interactive vizualization (**Map.addLayer**). To achieve desirable visualization effects you could use **ee.Image.getThumbURL** in replacement of Map.addLayer. Both  ee.Image.getThumbURL and Map.addLayer have the same parameters:



Parameter | Description | Type
--------------|-------------|-------------
bands	| Comma-delimited list of three band names to be mapped to RGB| list
min|	Value(s) to map to 0	number or list of three numbers, one for each |band
max|	Value(s) to map to 255	|number or list of three numbers, one for each band
gain|	Value(s) by which to multiply each pixel value|	number or list of three numbers, one for each band
bias|	Value(s) to add to each DN	| number or list of three numbers, one for each band
gamma|	Gamma correction factor(s)	| number or list of three numbers, one for each band
palette|	List of CSS-style color strings (single-band images only)	|comma-separated list of hex strings
opacity|	The opacity of the layer (0.0 is fully transparent and 1.0 is fully opaque)	| number
format|	Either "jpg" or "png"	|string

In [0]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Define the visualization parameters.
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

display_image(image.getThumbUrl(vizParams), 'San Francisco Bay')

### 4. Masking

You can use **`image.updateMask()`** to set the opacity of individual pixels based on where pixels in a mask image are non-zero. Pixels equal to zero in the mask are excluded from computations and the opacity is set to 0 for display. The following example uses an NDWI threshold (see the [Relational Operations section]() for information on thresholds) to update the mask on the NDWI layer created previously:



In [0]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B5'])

# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
ndwiViz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}


display_image(ndwiMasked.getThumbUrl(ndwiViz), 'NDWI < 0.4')

### 5. Clipping

The **`image.clip()`** method is useful for achieving cartographic effects. The following example clips the Image **RGB_SF**.

In [0]:
# Create a circle by drawing a 20000 meter buffer around a point.
roi = ee.Geometry.Point([-122.4481, 37.7599]).buffer(20000)

# Display a clipped version of the RGB image.
display_image(RGB_SF.clip(roi).getThumbUrl(vizParams), 'Clipped')

### 6. Rendering categorical maps

Palettes are also useful for rendering discrete valued maps, for example a land cover map. In the case of multiple classes, use the palette to supply a different color for each class. (The **`image.remap()`** method may be useful in this context, to convert arbitrary labels to consecutive integers). The following example uses a palette to render land cover categories:



In [0]:
cover = ee.Image('MODIS/051/MCD12Q1/2012_01_01')\
          .select('Land_Cover_Type_1')

# Define a palette for the 18 distinct land cover classes.
igbpPalette = [
  'aec3d4', # water
  '152106', '225129', '369b47', '30eb5b', '387242', # forest
  '6a2325', 'c3aa69', 'b76031', 'd9903d', '91af40',  # shrub, grass
  '111149', # wetlands
  'cdb33b', # croplands
  'cc0013', # urban
  '33280d', # crop mosaic
  'd7cdcc', # snow and ice
  'f7e084', # barren
  '6f6f6f'  # tundra
]

# Specify the min and max labels and the color palette matching the labels.
vizParams = {'min': 0,
             'max': 17,
             'palette': igbpPalette}

display_image(cover.getThumbUrl(vizParams),'Land Cover Distribution')

### 7. Image information and metadata

To explore image bands and properties in the Python API, getInfo() the image and inspect the output in the console. This information can also be accessed programmatically. For example, the following demonstrates how to access information about bands, projections and other metadata:




In [0]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# Get information about the bands as a list.
bandNames = image.bandNames()
print('Band names: ', bandNames.getInfo()) # ee.List of band names

In [0]:
# Get projection information from band 1.
b1proj = image.select('B1').projection()
print('Band 1 projection: \n', b1proj.getInfo()) # ee.Projection object

In [0]:
# Get scale (in meters) information from band 1.
b1scale = image.select('B1').projection().nominalScale()
print('Band 1 scale: \n', b1scale.getInfo()) # ee.Number

In [0]:
# Note that different bands can have different projections and scale.
b8scale = image.select('B8').projection().nominalScale()
print('Band 8 scale: \n', b8scale.getInfo()) # ee.Number

In [0]:
# Get a list of all metadata properties.
properties = image.propertyNames()
print('Metadata properties:', ) 
properties.getInfo() # ee.List of metadata properties

In [0]:
# Get a specific metadata property.
cloudiness = image.get('CLOUD_COVER')
print('CLOUD_COVER: ')
cloudiness.getInfo() # ee.Number

In [0]:
from datetime import datetime as dt

# Get the timestamp and convert it to a date.
date = ee.Date(image.get('system:time_start'))
# We divide by 1000 because Earth Engine returns the timestamp in milisecond and Python in seconds.
tmstp = date.getInfo()['value']/1000 

print('Timestamp:', dt.utcfromtimestamp(tmstp).strftime('%Y-%m-%d %H:%M:%S'))

Note that the results of these queries are **server-side objects**. When you use the method \*.getInfo(), you request that information describing the object be sent from the server to your client. (Learn more about client vs. server in Earth Engine on this [page](https://developers.google.com/earth-engine/client_server)). 



### 8. Mathematical Expressions

Earth Engine supports many basic mathematical operators. They share some common features. **Earth Engine performs math operations per pixel**. When an operator is applied to an image, it's applied to each unmasked pixel of each band. In the case of operations on two images, the operation is only applied at the locations where pixels in both images are unmasked. Earth Engine automatically matches bands between images. When an operator is applied to two images, the images are expected to have the same number of bands so they can be matched pairwise. However, if one of the images has only a single band, it is matched with all of the bands in the other image, essentially replicating that band enough times to match the other image.


For a simple example, consider the task of creating the Normalized Difference Vegetation Index (NDVI) using Landsat imagery:



In [0]:
# Load two 5-year Landsat 7 composites.
landsat1999 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
landsat2008 = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Compute NDVI the hard way.
ndvi1999 = landsat1999.select('B4')\
                      .subtract(landsat1999.select('B3'))\
                      .divide(landsat1999.select('B4').add(landsat1999.select('B3')))

# Compute NDVI the easy way.
ndvi2008 = landsat2008.normalizedDifference(['B4', 'B3'])

#Vizualization
ndwiViz = {'min': 0, 'max': 1, 'palette': ['FF0000', '00FF00']}
Peru = ee.Geometry.Rectangle(-85, -20, -65,0) 

img1 = display_image(ndvi1999.clip(Peru).getThumbUrl(ndwiViz),'NDVI PERU 1999')
img2 = display_image(ndvi2008.clip(Peru).getThumbUrl(ndwiViz),'NDVI PERU 2008')
display(img1, img2)

####  Expressions

To implement more complex mathematical expressions, it may be more convenient to use **`image.expression()`**, which parses a text representation of a math operation. The following example uses **`expression()`** to compute the Enhanced Vegetation Index (EVI):

In [0]:
# Load a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Compute the EVI using an expression.
evi = image.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': image.select('B5'),
      'RED': image.select('B4'),
      'BLUE': image.select('B2')})

Eviviz = {'min': 0, 'max': 1, 'palette': ['FF0000', '00FF00']}

display_image(ndvi1999.clip(Peru).getThumbUrl(Eviviz),'EVI 1999')

Observe that the first argument to expression is the textual representation of the math operation, the second argument is a dictionary where the keys are variable names used in the expression and the values are the image bands to which the variables should be mapped. Bands in the image may be referred to as `b("band name")` or `b(index)`, for example `b(0)`, instead of providing the dictionary. Note that division functions as it does in Python: dividing two integers results in an integer. For example `10 / 20 = 0`. To change this behavior, multiply one of the operands by `1.0: 10 * 1.0 / 20 = 0.5`. Supported expression operators are listed in the following table.

 _ | _ | Operators for expression()
--------------|-------------|-------------
Arithmetic	|+ - * / % **	|Add, Subtract, Multiply, Divide, Modulus, Exponent
Comparison	| == != < > <= >= |	Equal, Not Equal, Less Than, Greater than, etc.
Logical |	&& \|\| ! ^	| And, Or, Not, Xor
Ternary	| ? :	| If then else

### 9. Image Collection Overview

An `ImageCollection` is a **stack or time series of images**. In addition to loading an `ImageCollection` using an Earth Engine collection ID, Earth Engine has methods to create image collections. The constructor `ee.ImageCollection()` or the convenience method `ee.ImageCollection.fromImages()` create image collections from lists of images. You can also create new image collections by merging existing collections. For example:



In [0]:
# Create arbitrary constant images.
constant1 = ee.Image(1)
constant2 = ee.Image(2)

# Create a collection by giving a list to the constructor.
collectionFromConstructor = ee.ImageCollection([constant1, constant2])
print('collectionFromConstructor: ')
collectionFromConstructor.getInfo()

In [0]:
# Create a collection with fromImages().
collectionFromImages = ee.ImageCollection.fromImages([ee.Image(3), ee.Image(4)])
print('collectionFromImages: ')
collectionFromImages.getInfo()

In [0]:
# Merge two collections.
mergedCollection = collectionFromConstructor.merge(collectionFromImages)
print('mergedCollection: ')
mergedCollection.getInfo()

In [0]:
# Create a toy FeatureCollection
features = ee.FeatureCollection(
  [ee.Feature(None, {'foo': 1}), ee.Feature(None, {'foo': 2})])

# Create an ImageCollection from the FeatureCollection
# by mapping a function over the FeatureCollection.
images = features.map(lambda feature:ee.Image(ee.Number(feature.get('foo'))))

# Print the resultant collection.
print('Image collection: ')
images.getInfo()

Note that in this example an `ImageCollection` is created by mapping a function that returns an `Image` over a `FeatureCollection`. Learn more about mapping in the [Mapping over an ImageCollection section](https://developers.google.com/earth-engine/ic_mapping). Learn more about feature collections from the [FeatureCollection section](https://developers.google.com/earth-engine/feature_collections).

### 10 . ImageCollection Information and Metadata

As with Images, there are a variety of ways to get information about an ImageCollection. The collection can be printed directly to the console, but the console printout is **limited to 5000 elements**. Collections larger than 5000 images will need to be filtered before printing. Printing a large collection will be correspondingly slower. The following example shows various ways of getting information about image collections programmatically.

In [0]:
# Load a Landsat 8 ImageCollection for a single path-row.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
               .filter(ee.Filter.eq('WRS_PATH', 44))\
               .filter(ee.Filter.eq('WRS_ROW', 34))\
               .filterDate('2014-03-01', '2014-08-01')
print('Collection: ')
collection.getInfo()

In [0]:
# Get the number of images.
count = collection.size()
print('Count: ', count.getInfo())

In [0]:
# Get the date range of images in the collection.
rango = collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])

# Passing numeric date to standard
init_date = ee.Date(rango.get('min')).getInfo()['value']/1000.
init_date_f = dt.utcfromtimestamp(init_date).strftime('%Y-%m-%d %H:%M:%S')

last_date = ee.Date(rango.get('max')).getInfo()['value']/1000.
last_date_f = dt.utcfromtimestamp(last_date).strftime('%Y-%m-%d %H:%M:%S')

print('Date range: ',init_date_f,' - ',last_date_f)

In [0]:
# Get statistics for a property of the images in the collection.
sunStats = collection.aggregate_stats('SUN_ELEVATION')
print('Sun elevation statistics: ')
sunStats.getInfo()

In [0]:
# Sort by a cloud cover property, get the least cloudy image.
image = ee.Image(collection.sort('CLOUD_COVER').first())
print('Least cloudy image: ', )
image.getInfo()

In [0]:
# Limit the collection to the 10 most recent images.
recent = collection.sort('system:time_start', False).limit(10)
print('Recent images: ')
recent.getInfo()

<!--NAVIGATION-->
 < [Data structures in GEE](1.2_DataStructures.ipynb) | [Contents](index.ipynb) |  [Feature & FeatureCollections](1.4_Features.ipynb)>

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/1.3_Images.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>